In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import type_score,get_year_ipcc,get_year_ipbes,get_countries,check_doi_glutton,normalize,aplatir,remove_too_long
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache

In [ ]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [ ]:
pd.Series([list(pd.Series([x[i].get('wg') for i in range(len(x))]).drop_duplicates()) for x in list(df_ipcc.drop_duplicates(subset=['doi']).dropna(subset=['doi']).wg_chap)]).value_counts()

In [ ]:
min([int(x) for x in list(df_ipcc.year_OA.drop_duplicates().dropna())])

In [ ]:
df_ipcc.test_glutton.value_counts()

In [ ]:
df_ipcc=df_ipcc.rename(columns={'wg_chap':'ipcc'})

In [ ]:
df_ipcc.columns

In [ ]:
df_ipcc_group=df_ipcc.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0],'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0], 'ipcc': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0], 'freq':'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [ ]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [ ]:
len(df_ipcc_group.doi.drop_duplicates())==len(df_ipcc_group.doi)

In [ ]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [ ]:
df_ipbes.columns

In [ ]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: [{'chapter': str(x['tag']).replace('ipbes-global_chapter','')} for x in row['tags']], axis=1)

In [ ]:
df_ipbes.countries.value_counts()

In [ ]:
df_ipbes[df_ipbes.doi=='10.1016/j.cosust.2016.12.006'].countries[84]

In [ ]:
df_ipbes_group=df_ipbes.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0],'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0],'ipbes': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0]})
df_ipbes_group=df_ipbes_group.reset_index()

In [ ]:
df_ipbes_group[df_ipbes_group.doi=='10.1016/j.cosust.2016.12.006'].countries[909]

In [ ]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [ ]:
len(df_ipbes_group.doi.drop_duplicates())==len(df_ipbes_group.doi)

In [ ]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')
df_ipcc_ipbes=df_ipcc_ipbes[['doi', 'year_x', 'rors_x', 'authors_name_x', 'institutions_names_x',
       'ipcc', 'countries_x', 'ipbes']].rename(columns={'year_x':'year', 'rors_x':'rors','authors_name_x':'authors_name','institutions_names_x':'institutions_names', 'countries_x':'countries'})

In [ ]:
len(df_ipcc_ipbes.doi.drop_duplicates())==len(df_ipcc_ipbes.doi)

In [ ]:
update_bso_publications()

In [ ]:
bso=pd.DataFrame(get_bso_publications())

In [ ]:
df_bso=bso.transpose().reset_index()

In [ ]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [ ]:
df_bso

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left').rename(columns={'rors_x':'rors'})

In [ ]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

In [ ]:
del df_ipcc_ipbes_bso['rors_y']

In [ ]:
df_ipcc_ipbes_bso.columns

In [ ]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso_countries.json')

In [ ]:
df_iso

In [ ]:
dict_iso={}
for i,row in df_iso.iterrows():
    dict_iso[row['code']]=row['name']
dict_iso['CW']='Curaçao'
dict_iso['RS']='Serbia'
dict_iso['ME']='Montenegro'
dict_iso['XK']='Kosovo'
dict_iso['SS']='South Soudan'
dict_iso['MF']='Saint Martin'

In [ ]:
df_ipcc_ipbes_bso.loc[:,'countries']=df_ipcc_ipbes_bso.loc[:,'countries'].apply(lambda x: [dict_iso[y] for y in x] if (x!=None)&(x!=[])&(isinstance(x,float)==False) else None)

In [ ]:
df_brevet_doi=pd.read_json('brevet_doi.json')

In [ ]:
df_brevet_doi

In [ ]:
df_ipcc_ipbes_bso_brevet=pd.merge(df_ipcc_ipbes_bso,df_brevet_doi, on='doi', how='left')

In [ ]:
len(df_ipcc_ipbes_bso_brevet.doi.drop_duplicates())==len(df_ipcc_ipbes_bso_brevet.doi)

In [ ]:
df_ipcc_ipbes_bso_brevet.doi.value_counts()[:10]

In [ ]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

In [ ]:
df_cache

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [ ]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

In [ ]:
dict_doi_bso=[]

In [ ]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [ ]:
len(dict_doi_bso)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [ ]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes2.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [ ]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes2.jsonl', lines=True)

In [ ]:
dd.ipbes.value_counts()

In [ ]:
dd2=dd.dropna(subset=['ipcc']).drop_duplicates(subset=['doi'])

In [ ]:
len(dd2.loc[(dd2['ipcc'].apply(lambda wg_chap: any('1' in x.get('wg', '') for x in wg_chap)))])

In [ ]:
current_dict = dd.to_dict(orient='records')

In [ ]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [ ]:
df_elastic_input=pd.DataFrame(current_dict)

In [ ]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [ ]:
del df_elastic_input['openalex_abstract_inverted_index']
del df_elastic_input['openalex_primary_topic']

In [ ]:
df_elastic_input=df_elastic_input[['doi', 'year', 'rors', 'ipcc','authors_name', 'institutions_names',
       'countries', 'ipbes', 'openalex_authorships','openalex_topics']]

In [ ]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

In [2]:
df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [3]:
df.loc[:,'rors']=df.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'authors_name']=df.loc[:,'authors_name'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'institutions_names']=df.loc[:,'institutions_names'].apply(lambda x: [[]] if x==[None] else x)

In [4]:
df.loc[pd.isna(df['rors'])==False,'rors']=df.loc[pd.isna(df['rors'])==False,'rors'].apply(lambda x: [j.replace('https://ror.org/','') for j in list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates())])

In [5]:
df.loc[pd.isna(df['institutions_names'])==False,'institutions_names']=df.loc[pd.isna(df['institutions_names'])==False,'institutions_names'].apply(lambda x: list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates()))

In [8]:
df.loc[pd.isna(df['authors_name'])==False,'authors_name']=df.loc[pd.isna(df['authors_name'])==False,'authors_name'].apply(lambda x: list(pd.Series(aplatir([list(y.keys()) for y in x if isinstance(y,dict)])).drop_duplicates()))

In [10]:
df[['doi','rors','authors_name','institutions_names','ipcc','ipbes']]

,doi,rors,authors_name,institutions_names,ipcc,ipbes
0,10.1001/jama.2012.108940,"[05cf8a891, 00y4zzh67, 04mrb6c22]","[Tia Powell, Dan Hanfling, Lawrence O. Gostin]","[Albert Einstein College of Medicine, George W...","[{'name': 'wg2_chap_06', 'wg': '2', 'chap': 6}]",NaN
1,10.1001/jama.2014.13186,"[03e3qgk42, 01y2jtd41, 00cvxb145, 00a0jsq62]","[Jonathan A. Patz, Howard Frumkin, Tracey Holl...","[University of Wisconsin Health, University of...","[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7}]",NaN
2,10.1001/jama.2014.15715,"[03vek6s52, 04b6nzv94]","[Jennifer F. Bobb, Ziad Obermeyer, Yun Wang, F...","[Harvard University, Brigham and Women's Hospi...","[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7}]",NaN
3,10.1001/jama.2020.1097,"[05vzafd60, 02e5dc168]","[Alexandra Phelan, Rebecca Katz, Lawrence O. G...","[Georgetown University, Center for Global Health]","[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7}]",NaN
4,10.1001/jama.2020.15398,"[02dgjyy92, 023kj5402, 05qwgg493]","[James M. Shultz, Craig Fugate, Sandro Galea]","[University of Miami, Craig Technologies (Unit...","[{'name': 'wg2_chap_04', 'wg': '2', 'chap': 4}...",NaN
...,...,...,...,...,...,...
58235,10.7770/cuhso-v27n1-art1134,NaN,NaN,NaN,NaN,[{'chapter': '3'}]
58236,10.7930/j0x63jt0,NaN,NaN,NaN,NaN,[{'chapter': 'ipbes-global_glossary'}]
58237,10.9755/ejfa.v25i1.15376,NaN,NaN,NaN,NaN,[{'chapter': '3'}]
58238,10.9758/cpn.2015.13.3.239,NaN,NaN,NaN,NaN,[{'chapter': '2.3_ncp'}]


In [ ]:
x=df.authors_name[0]

In [7]:
df.authors_name[:200].value_counts()

authors_name
[[]]                                                                                                                                                                                                                                                         2
[{'Tia Powell': ['US']}, {'Dan Hanfling': ['US']}, {'Lawrence O. Gostin': []}]                                                                                                                                                                               1
[{'C. Adam Schlosser': ['US']}, {'Kenneth Strzepek': ['US']}, {'Xiang Gao': ['US']}, {'Charles Fant': ['US']}, {'Élodie Blanc': ['US']}, {'Sergey Paltsev': ['US']}, {'Henry D. Jacoby': ['US']}, {'John M. Reilly': ['US']}, {'Arthur Gueneau': ['US']}]    1
[{'Geoffrey Gebbie': ['US']}]                                                                                                                                                                                                 